# Simple Alfalfa Example

## Setup
1. [Alfalfa stack deployed](https://github.com/NREL/alfalfa/wiki/Deployment) locally with **at least two workers**
1. Create `small_office.zip` from `models`, `weather`, and `measures` folders and `small_office.osw`

## Notes
1. For Alfalfa v0.2.0, API reads and writes are implemented over the [Haystack API](https://project-haystack.org/doc/docHaystack/HttpApi), particularly the `read` and `pointWrite` endpoints.


In [ ]:
import os
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient

In [ ]:
# Create a new client
ac = AlfalfaClient(url='http://localhost')

In [ ]:
# Local paths to models for upload
model_paths = []

# all_models = range(0,317)
# for full experiment use range(0,317)
for i in range(0,2):
    model_paths.append(f'../CCTwin-scripts/bldg_models/apr22demo/zips/{i:03}.zip')

print(model_paths)

In [ ]:
# Upload to Alfalfa 
model_ids = {}
for p in model_paths:
    print(p)
    # TODO add error handling
    model_ids[p] = ac.submit(p)
# m1_id = ac.submit(m1)
# m2_id = ac.submit(m2)


In [ ]:
print(model_ids)

In [ ]:
# Define the run parameters.  
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2017, 9, 26, 0, 0, 0)
end_dt = datetime.datetime(2017, 9, 27, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": "true",
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

In [ ]:
# Start simulations.  since we're using external clock, will take through warmup and stop
for i in model_ids.values():
    print(i)
    ac.start(i, **params)

In [ ]:
#run for 1 day  
# timesteps = 1440
# limited timesteps during testing
timesteps = 2

# simulation loop
# TODO add error handling if expected vals aren't returned
for i in range(timesteps):
    aggregate_load_for_timestep = 0

    # Load current timestamp from one sim.  
    # note you could avoid an api call by adding 60 to start time each iteration
    t = ac.get_sim_time(list(model_ids.values())[0])

    # Load current values from each model
    for mid in model_ids.values():
        outputs = ac.outputs(mid)
        #print(outputs)

        # Note the key here is configured in the uploaded model
        main = outputs['Electric Meter']
        agg+= main

    # do somethinig with the aggregate demand across buildings for this timestep
    # note that outside this notebook we also are persisting all outputs by building to InfluxDB each timestep
    print(f"{t}, community aggregate: {agg}")

    # Advance the models.  
    # as currently implemented, there is some serial/blocking execution here at the web layer 
    # which may add up delays when run for 100s buildings
    ac.advance(model_ids.values())

In [ ]:
# Stop the models
for id in model_ids.values():
    ac.stop(id)